# Análisis Comparativo de Etiquetadores

Este notebook analiza los resultados de la comparación entre stanza y spacy para el etiquetado POS.

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown

## Carga y Preparación de Datos

In [2]:
import pandas as pd
from difflib import SequenceMatcher
from typing import List, Tuple, Dict

class TaggingAnalyzer:
    def __init__(self):
        # Términos identificados como potencialmente ambiguos
        self.terms_of_interest = ['back', 'run', 'will', 'like', 'bombing', 'up']
        
        # Cargar archivos de etiquetado (generados previamente por el evaluador)
        self._load_tagged_texts()
        self._load_metrics()
        
        # Generar DataFrame comparativo alineado para la comparación
        self.comparison_df = self.get_comparison_df()
    
    def _load_tagged_texts(self):
        """Carga y procesa los textos etiquetados desde archivos previamente generados."""
        def read_tagged_file(filename: str) -> pd.DataFrame:
            with open(filename, 'r', encoding='utf-8') as f:
                text = f.read().strip()
            # Separa cada par token/tag usando rsplit para casos en que el token contenga '/'
            pairs = []
            for pair in text.split():
                if '/' in pair:
                    token, tag = pair.rsplit('/', 1)
                    pairs.append((token, tag))
            return pd.DataFrame(pairs, columns=['token', 'tag'])
        
        self.gold_df = read_tagged_file('texto_ud_gold.txt')
        self.stanza_df = read_tagged_file('texto_stanza.txt')
        self.spacy_df = read_tagged_file('texto_spacy.txt')
    
    def _load_metrics(self):
        """Carga las métricas generadas por el evaluador."""
        self.spacy_metrics = pd.read_csv('spacy_metrics.csv')
        self.stanza_metrics = pd.read_csv('stanza_metrics.csv')
        self.results = pd.read_csv('resultados.csv')
    
    def align_tokens(self, source: List[str], target: List[str]) -> List[Tuple[int, int]]:
        """
        Alinea tokens entre la secuencia 'source' (por ejemplo, la salida de un tagger)
        y la secuencia 'target' (gold) utilizando SequenceMatcher.
        Retorna una lista de parejas (índice_source, índice_target).
        """
        s = SequenceMatcher(None, [t.lower() for t in source], [t.lower() for t in target])
        alignments = []
        for block in s.get_matching_blocks():
            i, j, n = block
            for k in range(n):
                alignments.append((i + k, j + k))
        return alignments
    
    def get_aligned_df(self, predicted_df: pd.DataFrame, tagger_name: str) -> pd.DataFrame:
        """
        Alinea los tokens del DataFrame 'predicted_df' con los tokens del gold.
        Retorna un DataFrame con las columnas:
            - 'Token': token del gold
            - 'Gold': etiqueta gold
            - Una columna con el nombre del tagger (p.ej. 'stanza' o 'spacy') con la etiqueta predicha.
        """
        gold_tokens = self.gold_df['token'].tolist()
        gold_tags = self.gold_df['tag'].tolist()
        predicted_tokens = predicted_df['token'].tolist()
        predicted_tags = predicted_df['tag'].tolist()
        
        alignments = self.align_tokens(predicted_tokens, gold_tokens)
        aligned_data = []
        for pred_idx, gold_idx in alignments:
            if pred_idx < len(predicted_tags) and gold_idx < len(gold_tags):
                aligned_data.append({
                    'Token': gold_tokens[gold_idx],
                    'Gold': gold_tags[gold_idx],
                    tagger_name: predicted_tags[pred_idx]
                })
        aligned_df = pd.DataFrame(aligned_data).reset_index(drop=True)
        return aligned_df
    
    def get_comparison_df(self) -> pd.DataFrame:
        """
        Genera un DataFrame comparativo alineando las salidas de stanza y spacy con el gold.
        Combina los resultados de ambos taggers en un único DataFrame.
        """
        stanza_aligned = self.get_aligned_df(self.stanza_df, 'stanza')
        spacy_aligned = self.get_aligned_df(self.spacy_df, 'spacy')
        
        # Se recorta al mínimo en caso de diferencias en la cantidad de tokens alineados
        min_length = min(len(stanza_aligned), len(spacy_aligned))
        stanza_aligned = stanza_aligned.iloc[:min_length].reset_index(drop=True)
        spacy_aligned = spacy_aligned.iloc[:min_length].reset_index(drop=True)
        
        # Combina los DataFrames (se usa el de stanza como base)
        comparison = stanza_aligned.copy()
        comparison['spacy'] = spacy_aligned['spacy']
        
        # Añade columnas que indican si la etiqueta predicha difiere del gold
        comparison['stanza_diff'] = comparison['Gold'] != comparison['stanza']
        comparison['spacy_diff'] = comparison['Gold'] != comparison['spacy']
        comparison['both_diff'] = comparison['stanza_diff'] & comparison['spacy_diff']
        
        return comparison
    
    def analyze_ambiguous_terms(self) -> pd.DataFrame:
        """
        Analiza los términos de interés en el DataFrame alineado.
        Añade una columna 'Context' con dos tokens antes y dos después del término.
        Retorna un DataFrame con los casos encontrados.
        """
        ambiguous_cases = []
        for term in self.terms_of_interest:
            term_rows = self.comparison_df[self.comparison_df['Token'].str.lower() == term].copy()
            if not term_rows.empty:
                contexts = []
                for idx in term_rows.index:
                    start = max(0, idx - 2)
                    end = min(len(self.comparison_df), idx + 3)
                    context = ' '.join(self.comparison_df.loc[start:end, 'Token'])
                    contexts.append(context)
                term_rows['Context'] = contexts
                ambiguous_cases.append(term_rows)
        if ambiguous_cases:
            return pd.concat(ambiguous_cases).reset_index(drop=True)
        return pd.DataFrame()
    
    def get_summary_stats(self) -> pd.DataFrame:
        """
        Genera estadísticas resumidas a partir del DataFrame comparativo alineado.
        Calcula, por ejemplo, la cantidad total de tokens y la precisión para cada tagger.
        """
        comp = self.comparison_df
        stats = {
            'Total tokens': len(comp),
            'stanza correct': (comp['Gold'] == comp['stanza']).sum(),
            'spacy correct': (comp['Gold'] == comp['spacy']).sum(),
            'both correct': ((comp['Gold'] == comp['stanza']) & (comp['Gold'] == comp['spacy'])).sum(),
        }
        total = stats['Total tokens']
        stats.update({
            'stanza accuracy %': (stats['stanza correct'] / total) * 100,
            'spacy accuracy %': (stats['spacy correct'] / total) * 100,
            'both accuracy %': (stats['both correct'] / total) * 100
        })
        stats_df = pd.DataFrame(stats, index=[0]).T.reset_index()
        stats_df.columns = ['Metric', 'Value']
        return stats_df
    
    def calculate_quantitative_metrics(self) -> Dict:
        """
        Calcula métricas cuantitativas detalladas.
        Retorna un diccionario con la precisión global y el top 10 de categorías por frecuencia,
        usando claves en minúsculas para mantener consistencia.
        """
        global_metrics = {
            'stanza': self.stanza_metrics['Correct'].sum() / self.stanza_metrics['Total'].sum(),
            'spacy': self.spacy_metrics['Correct'].sum() / self.spacy_metrics['Total'].sum()
        }
        
        # Top 10 categorías por frecuencia
        stanza_top = self.stanza_metrics.sort_values('Total', ascending=False).head(10)
        spacy_top = self.spacy_metrics.sort_values('Total', ascending=False).head(10)
        
        return {
            'global_metrics': global_metrics,
            'stanza_detailed': stanza_top,
            'spacy_detailed': spacy_top  # clave ahora en minúsculas: "spacy_detailed"
        }
    
    def display_quantitative_results(self):
        """Muestra los resultados cuantitativos en formato tabular."""
        metrics = self.calculate_quantitative_metrics()
        
        print("Precisión global:")
        for tagger, acc in metrics['global_metrics'].items():
            print(f"{tagger}: {acc:.2%}")
        
        print("\nPrecisión por categoría (top 10 más frecuentes):")
        print("\nstanza:")
        display(metrics['stanza_detailed'].style.format({'Accuracy': '{:.2%}'}))
        print("\nspacy:")
        display(metrics['spacy_detailed'].style.format({'Accuracy': '{:.2%}'}))



## Análisis de Diferencias en el Etiquetado

In [3]:
analyzer = TaggingAnalyzer()

# Mostrar estadísticas generales
display(Markdown("### Estadísticas Generales"))
stats_df = analyzer.get_summary_stats()
display(stats_df.style.format({
    'Value': '{:.2f}'
}))


### Estadísticas Generales

,Metric,Value
0,Total tokens,297.00
1,stanza correct,294.00
2,spacy correct,289.00
3,both correct,288.00
4,stanza accuracy %,98.99
5,spacy accuracy %,97.31
6,both accuracy %,96.97


## Análisis de Términos Ambiguos

In [4]:
# Mostrar análisis de términos ambiguos
ambiguous_df = analyzer.analyze_ambiguous_terms()
if not ambiguous_df.empty:
    display(Markdown("### Casos de Ambigüedad Encontrados"))
    display(ambiguous_df.style.highlight_null()
           .hide(axis='index')  # Cambiado de hide_index() a hide(axis='index')
           .set_properties(**{'text-align': 'left'}))

### Casos de Ambigüedad Encontrados

Token,Gold,stanza,spacy,stanza_diff,spacy_diff,both_diff,Context
back,RB,RB,RB,False,False,False,Weathermen bombers back in the 1960s
run,VBN,VBN,VBN,False,False,False,were being run by 2 officials
run,VBN,VBN,VBN,False,False,False,was being run by the head
will,MD,MD,MD,False,False,False,respected cleric will be causing us
like,IN,IN,IN,False,False,False,would be like having J. Edgar
bombing,NN,NN,NN,False,False,False,with his bombing targets . The
up,RP,RP,RP,False,False,False,had busted up 3 terrorist cells
up,RP,RP,RP,False,False,False,"and breaking up terror cells ,"


## Visualización de Diferencias

In [5]:
# Obtener todas las diferencias usando los nombres en minúsculas
comparison_df = analyzer.get_comparison_df()
differences = comparison_df[comparison_df['stanza_diff'] | comparison_df['spacy_diff']]

display(Markdown("### Tokens con Diferencias en el Etiquetado"))
display(differences[['Token', 'Gold', 'stanza', 'spacy']]
        .head(15)
        .style.highlight_null()
        .hide(axis='index')
        .set_properties(**{'text-align': 'left'}))

### Tokens con Diferencias en el Etiquetado

Token,Gold,stanza,spacy
respected,JJ,VBN,JJ
DPA,NNP,NNP,XX
MoI,NNP,NNP,NN
so,RB,RB,CC
employ,VB,VB,NN
Weathermen,NNPS,NNPS,NNP
do,VBP,VB,VB
much,RB,JJ,JJ
Guerrillas,NNS,NNS,NNP


## Generación de Tablas para LaTeX

In [6]:
def generate_latex_table(df, caption):
    """Genera código LaTeX para una tabla"""
    latex = df.to_latex(index=False, 
                        caption=caption,
                        escape=False,
                        column_format='l' * len(df.columns))
    return latex

# Generar tabla de diferencias para LaTeX
latex_diff_table = generate_latex_table(
    differences[['Token', 'Gold', 'stanza', 'spacy']].head(10),
    'Diferencias significativas en el etiquetado'
)

# Mostrar el código LaTeX
print(latex_diff_table)

\begin{table}
\caption{Diferencias significativas en el etiquetado}
\begin{tabular}{llll}
\toprule
Token & Gold & stanza & spacy \\
\midrule
respected & JJ & VBN & JJ \\
DPA & NNP & NNP & XX \\
MoI & NNP & NNP & NN \\
so & RB & RB & CC \\
employ & VB & VB & NN \\
Weathermen & NNPS & NNPS & NNP \\
do & VBP & VB & VB \\
much & RB & JJ & JJ \\
Guerrillas & NNS & NNS & NNP \\
\bottomrule
\end{tabular}
\end{table}



In [7]:
# Resultados cuantitativos para la sección 3.2
display(Markdown("## 3.2 Evaluación cuantitativa"))

analyzer = TaggingAnalyzer()
metrics = analyzer.calculate_quantitative_metrics()

# Crear tabla de precisión global
global_df = pd.DataFrame(metrics['global_metrics'].items(), 
                        columns=['Etiquetador', 'Precisión'])
print("\nPrecisión global de los etiquetadores:")
display(global_df.style.format({'Precisión': '{:.2%}'})
        .hide(axis='index')
        .set_properties(**{'text-align': 'left'}))

# Crear tablas comparativas por categoría
print("\nPrecisión por categoría gramatical (top 10 más frecuentes):")
for tagger, df in [('stanza', metrics['stanza_detailed']), 
                  ('spacy', metrics['spacy_detailed'])]:
    print(f"\n{tagger}:")
    display(df.style.format({'Accuracy': '{:.2%}'})
           .hide(axis='index')
           .set_properties(**{'text-align': 'left'}))

# Generar código LaTeX para las tablas
latex_global = global_df.to_latex(index=False, 
                                caption='Precisión global de los etiquetadores',
                                label='tab:precision-global',
                                float_format=lambda x: '{:.2%}'.format(x))

print("\nCódigo LaTeX para la tabla de precisión global:")
print(latex_global)

## 3.2 Evaluación cuantitativa


Precisión global de los etiquetadores:


Etiquetador,Precisión
stanza,98.99%
spacy,97.31%



Precisión por categoría gramatical (top 10 más frecuentes):

stanza:


Gold,Total,Correct,Accuracy
IN,44,44,100.00%
NNP,36,36,100.00%
DT,33,33,100.00%
NN,30,30,100.00%
NNS,18,18,100.00%
VBD,16,16,100.00%
.,14,14,100.00%
JJ,13,12,92.31%
VBG,11,11,100.00%
PRP,11,11,100.00%



spacy:


Gold,Total,Correct,Accuracy
IN,44,44,100.00%
NNP,36,34,94.44%
DT,33,33,100.00%
NN,30,30,100.00%
NNS,18,17,94.44%
VBD,16,16,100.00%
.,14,14,100.00%
JJ,13,13,100.00%
VBG,11,11,100.00%
PRP,11,11,100.00%



Código LaTeX para la tabla de precisión global:
\begin{table}
\caption{Precisión global de los etiquetadores}
\label{tab:precision-global}
\begin{tabular}{lr}
\toprule
Etiquetador & Precisión \\
\midrule
stanza & 98.99% \\
spacy & 97.31% \\
\bottomrule
\end{tabular}
\end{table}



In [8]:
with open('texto_ud.txt', 'r') as f:
    texto = f.read()
    tokens_originales = len(texto.split())
print(f"Tokens en texto original: {tokens_originales}")

Tokens en texto original: 298


In [9]:
with open('texto_ud_gold.txt', 'r') as f:
    gold = f.read()
    tokens_gold = len([p for p in gold.split() if '/' in p])
print(f"Tokens en gold standard: {tokens_gold}")

Tokens en gold standard: 298


In [10]:
def analyze_alignment():
    analyzer = TaggingAnalyzer()
    
    # Contar tokens en cada fuente
    gold_count = len(analyzer.gold_df)
    stanza_count = len(analyzer.stanza_df) 
    spacy_count = len(analyzer.spacy_df)
    
    print(f"Tokens en gold: {gold_count}")
    print(f"Tokens en stanza: {stanza_count}")
    print(f"Tokens en spacy: {spacy_count}")
    
    # Encontrar diferencias en tokenización
    gold_tokens = set(analyzer.gold_df['token'])
    stanza_tokens = set(analyzer.stanza_df['token'])
    spacy_tokens = set(analyzer.spacy_df['token'])
    
    print("\nTokens únicos en gold:")
    print(gold_tokens - stanza_tokens)
    print("\nTokens únicos en stanza:")
    print(stanza_tokens - gold_tokens)
    print("\nTokens únicos en spacy:")
    print(spacy_tokens - gold_tokens)

analyze_alignment()

Tokens en gold: 298
Tokens en stanza: 299
Tokens en spacy: 299

Tokens únicos en gold:
{"don't"}

Tokens únicos en stanza:
set()

Tokens únicos en spacy:
set()
